Group: I-Need-More-Boolets

Members: <br>
Go, Ryan Jefferson <br>
Ejercito, Joshua Carl <br>
Moraña, Anton Louis <br>
Nieva, Samuel 

Section: S11

# Introduction to the problem/task and dataset

The groups has selected the stars dataset to create three classification models specifically (will put soon) will be used to for stellar classification. Stellar classification is the classification of stars based on their spectral 
characteristics The target of the each regression model is to predict the class of the entry. The class could be either be a quasar, star or galaxy>. 

# Description of the dataset

The Stars datasets contains spectral characterstics that is then classified into either a quassar, star, or galaxy. The datasets is made up of 100,000 obersavations of space taken by the SDSS (Sloan Digital Sky Survey) with each observations having 18 features. 


